In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce
import itertools
from collections import Counter

In [ ]:
# dodac produkcje zwracaja nowe pod grafy <check> 
# dodac do node'ow typu I id parenta, pierwszy node Typu I bedzie zawierac parenta -1 <check>
# layer ma byc lista <check>
# wizualizacja warstwy (brutem) <check>
# dodac check czy mozna wywolac <check>
#czyscic kernel i out przed pushem

In [ ]:
class Id_creator:
    def __init__(self):
        self.last_id = -1
    def get_id(self):
        self.last_id +=1
        return self.last_id
    def __call__(self):
        return self.get_id()
    
    
class CannotExecuteProduction(Exception):
        pass
    
    
class Graph_layers:
    def __init__(self):
        self._node_id_gen =  Id_creator()
        G = nx.Graph()
        G.add_nodes_from([(self._node_id_gen(), {'pos': (0,0),'type': 'e'})])
        self._layers=[[G]]
        
    def get_layer(self, i):
        return self._layers[i]
    
    def add_to_layer(self,i,G):
        return self._layers[i].append(G)
    
    def add_to_last_layer(self,G):
        return self._layers[-1].append(G)
        
    def add_new_layer(self, G_new):
        self._layers.append([G_new])            
        
    def get_last_layer_index(self):
        return len(self._layers) - 1
    
    def get_last_layer(self,i):
        return self._layers[i]
    
    def display_i_layer(self, i):
        G_layer = self._layers[i]
        G = self._layers[i][0] if  len(self._layers[i])==1 else reduce(nx.algorithms.operators.binary.compose,G_layer)
        
        pos = nx.get_node_attributes(G, 'pos')
        nx.draw_networkx(G, pos)
        
    def get_node_id_gen(self):
        return self._node_id_gen

In [ ]:
def p1(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
    assert(G.nodes[base_node_id]['type'].lower() == 'e' )
    assert(len(G.nodes)==1)
    all_new_nodes = []
    all_new_edges = []
    base_pos = G.nodes[base_node_id]['pos']
    x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
    i_node_x, i_node_y = base_pos[0], base_pos[1]
    i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
    all_new_nodes.append(i_node)
    
    half_side_len = side_len/2
    e_nodes = [
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
        (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'})
    ]
    for i in range(len(e_nodes)):
        all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
        all_new_edges.extend([(i_node[0],e_nodes[i][0])])
    
    all_new_nodes.extend(e_nodes)
    nG = nx.Graph()
    nG.add_nodes_from(all_new_nodes)
    nG.add_edges_from(all_new_edges)
    return nG, i_node


In [ ]:
def conc_duplicates(G):
    org_nodes = list(G.nodes(data=True))

    org_nodes_dict = {}
    for node_id,data in org_nodes:
        try:
            org_nodes_dict[data['pos']].append(node_id)
        except KeyError:
            org_nodes_dict[data['pos']]=[node_id]

    nodes_to_replace = {} #keys nodes to be deleted, values nodes to replace them
    for key,ids in org_nodes_dict.items():
        if len(ids) > 1:
            for id_ in ids[1:]:
                nodes_to_replace[id_] = ids[0]
                
    for old_node, new_node in nodes_to_replace.items():
        neighbors = list(G.neighbors(old_node))
        for n in neighbors:
            G.remove_edge(old_node,n)
            G.add_edge(new_node,n)
        G.remove_node(old_node)
    return G

In [ ]:
def avg_pos(pos1, pos2):
    return ((pos1[0] + pos2[0])/2, (pos1[1] + pos2[1])/2)

def rm_edge_if_exists(G,n1id,n2id):
    try:
        G.remove_edge(n1id,n2id)
    except nx.NetworkXError:
        pass
    


#     e0 - - - e1
#      | \    / |
#      |  I0    |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0 - - - n0 - - - e1
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    n1 - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2 - - - n4 - - - e3

def p2(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    if len(es) != 4:
        raise CannotExecuteProduction
    e0, e1, e2, e3 = None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    if not(G.has_edge(e0[0],e1[0]) and\
            G.has_edge(e1[0],e3[0]) and\
            G.has_edge(e3[0],e2[0]) and\
            G.has_edge(e0[0],e2[0]) and\
            G.has_edge(e0[0],base_node_id) and\
            G.has_edge(e1[0],base_node_id) and\
            G.has_edge(e2[0],base_node_id) and\
            G.has_edge(e3[0],base_node_id)\
            ):
        raise CannotExecuteProduction
    #prepare all new verticies accord to map above
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG
#     conc_duplicates(G)


In [ ]:
graph_layers = Graph_layers()
graph_layers.display_i_layer(0)

In [ ]:
base_node = list(graph_layers.get_layer(0)[0].nodes(data=True))[0]
#indexing from 0 
first_layer_G = graph_layers.get_layer(0)[0].copy()

#apply p1 production
G, i_node = p1(first_layer_G, base_node[0], graph_layers.get_node_id_gen()) 
graph_layers.add_new_layer(G)

graph_layers.display_i_layer(1)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(1)[0]
nG = p2(G, 1, graph_layers.get_node_id_gen())
graph_layers.add_new_layer(nG)
graph_layers._layers[2][0].nodes(data=True)
graph_layers.display_i_layer(2)

In [ ]:
#apply p2 production 4 times
G = graph_layers.get_layer(2)[0]
graph_layers.add_new_layer( p2(G, 15, graph_layers.get_node_id_gen()))
graph_layers.add_to_last_layer(p2(G, 18, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(2)[0]
graph_layers.add_to_last_layer(p2(G, 17, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
#apply p2 production
G = graph_layers.get_layer(2)[0]
graph_layers.add_to_last_layer(p2(G, 16, graph_layers.get_node_id_gen()))
graph_layers.display_i_layer(3)

In [ ]:
# obraz pogladowy jasne jest ze istnieja 4 permutacje izomorficzne jesli chodzi o labelkowanie wierzcholkow
#     e0 - - - e1
#      | \    / |
#     e4   I0   |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0' - - - n0 - - - e1'
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    e4' - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2' - - - n4 - - - e3'
def p3(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    nghs_counter = Counter(reduce(lambda a,b : a+b ,[list(G.neighbors(e)) for e in es],es + [base_node_id]))
    es1 = set([x for x, count in nghs_counter.items() if count >= 2]) - {base_node_id}
    if len(es) != 4 and len(es1) != 5:
        raise CannotExecuteProduction
    e0, e1, e2, e3, e4 = None, None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    
    
    # get Square corners
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    ids = [base_node_id, e0[0], e1[0], e2[0], e3[0]]

    edge0 = G.has_edge(e0[0],e1[0])
    edge1 = G.has_edge(e0[0],e2[0])
    edge2 = G.has_edge(e2[0],e3[0])
    edge3 = G.has_edge(e1[0],e3[0])
    
    # should be 3 edges between 4 cornesrs (1 is created with extra node)
    if(int(edge0) + int(edge1) + int(edge2) + int(edge3) != 3):
         raise CannotExecuteProduction
           
    def getNodeBetween(e1Id, e2Id):
        allNodes = list(G.neighbors(e1Id)) + list(G.neighbors(e2Id))
        possibleNodes = [n for n in allNodes if n in list(G.neighbors(e1Id)) and n in list(G.neighbors(e2Id))]
        return list(set(possibleNodes))
    
    if(not edge0):
        possE4 = getNodeBetween(e0[0],e1[0])
    if(not edge1):
        possE4 = getNodeBetween(e0[0],e2[0])
    if(not edge2):
        possE4 = getNodeBetween(e2[0],e3[0])
    if(not edge3):
        possE4 = getNodeBetween(e1[0],e3[0])
 
    # get all 'e' nodes between two nodes that does not share edge
    maybeE4 = [ G.nodes[e] for e in possE4 if G.nodes[e]['type'].lower() == 'e']
    if(len(maybeE4) != 1):
        raise CannotExecuteProduction
    
    # this is the extra node
    e4 = maybeE4[0]

    #check if node has proper cordinates
    if(not edge0):
        if(e4['pos'] != avg_pos(e0[1]['pos'], e1[1]['pos'])):
            raise CannotExecuteProduction
    if(not edge1):
        if(e4['pos'] != avg_pos(e0[1]['pos'], e2[1]['pos'])):
            raise CannotExecuteProduction
    if(not edge2):
        if(e4['pos'] != avg_pos(e2[1]['pos'], e3[1]['pos'])):
            raise CannotExecuteProduction
    if(not edge3):
        if(e4['pos'] != avg_pos(e1[1]['pos'], e3[1]['pos'])):
            raise CannotExecuteProduction
                                                                                     
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG


In [ ]:
# obraz pogladowy jasne jest ze istnieja 4 permutacje izomorficzne jesli chodzi o labelkowanie wierzcholkow
#     e0 - e5 - e1
#      | \    / |
#     e4   I0   |
#      | /   \  |
#     e2 - - - e3
#
#          |
#         \/
#
#     e0' - - - n0 - - - e1'
#     | \    / | \    / |
#     |  I1    |  I2    |
#     | /   \  | /   \  |
#    e4' - - - n2 - - - n3
#     | \    / | \    / |
#     |  I3    |  I4    |
#     | /   \  | /   \  |
#    e2' - - - n4 - - - e3'
def p4(G, base_node_id, n_id_gen):
    assert(G.nodes[base_node_id]['type'].lower() == 'i')
    nG = nx.Graph()
    es = list(G.neighbors(base_node_id))
    nghs_counter = Counter(reduce(lambda a,b : a+b ,[list(G.neighbors(e)) for e in es],es + [base_node_id]))
    es1 = set([x for x, count in nghs_counter.items() if count >= 2]) - {base_node_id}
    if len(es) != 4 and len(es1) != 6:
        raise CannotExecuteProduction
    e0, e1, e2, e3, e4, e5 = None, None, None, None, None, None
    baseX, baseY = G.nodes[base_node_id]['pos']
    
    
    # get Square corners
    for ex in es:
        x,y = G.nodes[ex]['pos']
        if x<baseX:
            if y<baseY:
                e2= (ex,G.nodes[ex])
            if y>baseY:
                e0= (ex,G.nodes[ex])
        else:
            if y<baseY:
                e3= (ex,G.nodes[ex])
            if y>baseY:
                e1= (ex,G.nodes[ex])
    ids = [base_node_id, e0[0], e1[0], e2[0], e3[0]]

    edge0 = G.has_edge(e0[0],e1[0])
    edge1 = G.has_edge(e0[0],e2[0])
    edge2 = G.has_edge(e2[0],e3[0])
    edge3 = G.has_edge(e1[0],e3[0])
    
    # should be 3 edges between 4 cornesrs (1 is created with extra node)
    if(int(edge0) + int(edge1) + int(edge2) + int(edge3) != 2):
         raise CannotExecuteProduction
    # 'edges' with addtional node may lay near each other (cannot be on the other side of square)
    if(int(edge0) + int(edge2) == 2):
        raise CannotExecuteProduction
    if(int(edge1) + int(edge3) == 2):
        raise CannotExecuteProduction
           
    def getNodeBetween(e1Id, e2Id):
        allNodes = list(G.neighbors(e1Id)) + list(G.neighbors(e2Id))
        possibleNodes = [n for n in allNodes if n in list(G.neighbors(e1Id)) and n in list(G.neighbors(e2Id))]
        return list(set(possibleNodes))
    
    possE4andE5 = list([])
    if(not edge0):
        possE4andE5 = possE4andE5 + getNodeBetween(e0[0],e1[0])
    if(not edge1):
        possE4andE5 = possE4andE5 + getNodeBetween(e0[0],e2[0])
    if(not edge2):
        possE4andE5 = possE4andE5 + getNodeBetween(e2[0],e3[0])
    if(not edge3):
        possE4andE5 = possE4andE5 + getNodeBetween(e1[0],e3[0])
 
    # get all 'e' nodes between two nodes that does not share edge
    maybeE4AndE5 = [ G.nodes[e] for e in possE4andE5 if G.nodes[e]['type'].lower() == 'e']
    if(len(maybeE4AndE5) != 2):
        raise CannotExecuteProduction
    
    # these are the extra nodes
    e4 = maybeE4AndE5[0]
    e5 = maybeE4AndE5[0]
    
    def checkIfHasOneOfProperPositionsOrRaise(e):
        proper = False
        if(not edge0):
            if(e['pos'] == avg_pos(e0[1]['pos'], e1[1]['pos'])):
                proper = True
        if(not edge1):
            if(e['pos'] != avg_pos(e0[1]['pos'], e2[1]['pos'])):
                proper = True
        if(not edge2):
            if(e['pos'] != avg_pos(e2[1]['pos'], e3[1]['pos'])):
                proper = True
        if(not edge3):
            if(e['pos'] != avg_pos(e1[1]['pos'], e3[1]['pos'])):
                proper = True
        if(not proper):
            raise CannotExecuteProduction
        
    
    #check if node has proper cordinates
    checkIfHasOneOfProperPositionsOrRaise(e4)
    checkIfHasOneOfProperPositionsOrRaise(e5)
    
    e0 = (n_id_gen(),{'pos': e0[1]['pos'],'type' : e0[1]['type']})
    e1 = (n_id_gen(),{'pos': e1[1]['pos'],'type' : e1[1]['type']})
    e2 = (n_id_gen(),{'pos': e2[1]['pos'],'type' : e2[1]['type']})
    e3 = (n_id_gen(),{'pos': e3[1]['pos'],'type' : e3[1]['type']})

    n0 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e1[1]['pos']),'type':'e'})
    n1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],e2[1]['pos']),'type':'e'})
    n2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e2[1]['pos']),'type':'e'})
    n3 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],e3[1]['pos']),'type':'e'})
    n4 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],e3[1]['pos']),'type':'e'})
    
    I1 = (n_id_gen(),{'pos' : avg_pos(e0[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I2 = (n_id_gen(),{'pos' : avg_pos(e1[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I3 = (n_id_gen(),{'pos' : avg_pos(e2[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    I4 = (n_id_gen(),{'pos' : avg_pos(e3[1]['pos'],n2[1]['pos']),'type':'I','parent':base_node_id})
    # add all new edges
    new_edges = [
        (e0[0],n0[0]),(n0[0],e1[0]),
        (e0[0],I1[0]),(n0[0],I1[0]), (n0[0],I2[0]),(e1[0],I2[0]),
        (e0[0],n1[0]),(n0[0],n2[0]),(e1[0],n3[0]),
        (n1[0],I1[0]),(n2[0],I1[0]), (n2[0],I2[0]),(n3[0],I2[0]),
        (n1[0],n2[0]),(n2[0],n3[0]),
        (n1[0],I3[0]),(n2[0],I3[0]), (n2[0],I4[0]),(n3[0],I4[0]),
        (n1[0],e2[0]),(n2[0],n4[0]),(n3[0],e3[0]),
        (e2[0],I3[0]),(n4[0],I3[0]), (n4[0],I4[0]),(e3[0],I4[0]),
        (e2[0],n4[0]),(n4[0],e3[0]),
    ]
    nG.add_nodes_from([e0,e1,e2,e3,n0,n1,n2,n3,n4,I1,I2,I3,I4])
    nG.add_edges_from(new_edges)
    return nG

In [ ]:
def zad_2_tests():
    
    def print_res(graph_layers,prod_f, prod,start_layer):
        fig1 = plt.figure(1)
        graph_layers.display_i_layer(start_layer)
        fig1.suptitle(f'base {prod} graph')
        plt.show()
        print(f'Applying {prod} production')
        graph_layers.add_new_layer(prod_f())
        fig2 = plt.figure(2)
        fig2.suptitle(f'production {prod} result')
        graph_layers.display_i_layer(start_layer+1)
        plt.show()
        print('Success')
        print()
    
    def prepare_graph(f=None):
        graph_layers = Graph_layers()
        base_node = list(graph_layers.get_layer(0)[0].nodes(data=True))[0]
        first_layer_G = graph_layers.get_layer(0)[0].copy()
        if f:
            G, i_node = f(first_layer_G, base_node[0], graph_layers.get_node_id_gen())
            graph_layers.add_new_layer(G)
        else:
            G, i_node = p1(first_layer_G, base_node[0], graph_layers.get_node_id_gen()) 
            graph_layers.add_new_layer(G)
            G = graph_layers.get_layer(1)[0]
            G = p2(G, 1, graph_layers.get_node_id_gen())
            graph_layers.add_new_layer(G)
            graph_layers._layers[2][0].nodes(data=True)
        return graph_layers,G,i_node
        
        
    def pX1(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
        assert(G.nodes[base_node_id]['type'].lower() == 'e' )
        assert(len(G.nodes)==1)
        all_new_nodes = []
        all_new_edges = []
        base_pos = G.nodes[base_node_id]['pos']
        x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        i_node_x, i_node_y = base_pos[0], base_pos[1]
        i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
        all_new_nodes.append(i_node)

        half_side_len = side_len/2
        e_nodes = [
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y),'type': 'e'})
        ]
        for i in range(len(e_nodes)):
            all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
            if i < 4:
                all_new_edges.extend([(i_node[0],e_nodes[i][0])])

        all_new_nodes.extend(e_nodes)
        nG = nx.Graph()
        nG.add_nodes_from(all_new_nodes)
        nG.add_edges_from(all_new_edges)
        return nG, i_node
    
    def pX2(G, base_node_id, n_id_gen, side_len=2, max_random_offset = 0):
        assert(G.nodes[base_node_id]['type'].lower() == 'e' )
        assert(len(G.nodes)==1)
        all_new_nodes = []
        all_new_edges = []
        base_pos = G.nodes[base_node_id]['pos']
        x_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        y_offset = ((np.random.random()-0.5) * max_random_offset * 2)
        i_node_x, i_node_y = base_pos[0], base_pos[1]
        i_node = (n_id_gen(), {'pos': (i_node_x+x_offset, i_node_y-y_offset), 'type': 'I','parent': -1})
        all_new_nodes.append(i_node)

        half_side_len = side_len/2
        e_nodes = [
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (7, {'pos': (i_node_x, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y + half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x + half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (n_id_gen(), {'pos': (i_node_x - half_side_len, i_node_y - half_side_len),'type': 'e'}),
            (6, {'pos': (i_node_x - half_side_len, i_node_y),'type': 'e'})
        ]
        for i in range(len(e_nodes)):
            all_new_edges.extend([(e_nodes[i][0],e_nodes[i+1 if i + 1 < len(e_nodes) else 0][0])])
            if i != 1 and i != 5:
                all_new_edges.extend([(i_node[0],e_nodes[i][0])])

        all_new_nodes.extend(e_nodes)
        nG = nx.Graph()
        nG.add_nodes_from(all_new_nodes)
        nG.add_edges_from(all_new_edges)
        return nG, i_node
    
    #TC1
    print('p3 simple graph')
    graph_layers, G, i_node = prepare_graph(pX1)
    
    print("Trying to use p4 on p3 graph")
    try:
        p4(G, i_node[0], graph_layers.get_node_id_gen()) 
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")

    print_res(graph_layers,lambda :p3(G, i_node[0], graph_layers.get_node_id_gen()),'p3',1)
    
    #TC2
    print('p4 simple graph')
    graph_layers, G, i_node = prepare_graph(pX2)
    
    print("Trying to use p3 on p4 graph")
    try:
        p3(G, i_node[0], graph_layers.get_node_id_gen()) 
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")
        
    print_res(graph_layers,lambda :p4(G, i_node[0], graph_layers.get_node_id_gen()), 'p4',1)
    
    #TC3
    print('p3 not so simple graph')
    graph_layers, G, i_node = prepare_graph()
    rm_edge_if_exists(G, 6, 11)
    rm_edge_if_exists(G, 6, 10)
    n6 = list(G.nodes(data=True))[0]
    n10 = list(G.nodes(data=True))[4]
    n11 = list(G.nodes(data=True))[5]
    n19 = (19, {'pos' : avg_pos(n6[1]['pos'],n11[1]['pos']),'type':'e'})
    n20 = (20, {'pos' : avg_pos(n6[1]['pos'],n10[1]['pos']),'type':'e'})
    G.add_nodes_from([n19, n20])
    G.add_edges_from([(n6[0], n19[0]), (n19[0], n11[0]), (n20[0], n6[0]), (n10[0], n20[0])])
    
    print("Trying to use p3 on p4 graph")
    try:
        p3(G, 15, graph_layers.get_node_id_gen())
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")
    
    print_res(graph_layers,lambda :p4(G, 15, graph_layers.get_node_id_gen()),'p4',2)
    
    #TC4
    print('p4 not so simple graph')
    graph_layers, G, i_node = prepare_graph()
    rm_edge_if_exists(G, 6, 11)
    n6 = list(G.nodes(data=True))[0]
    n11 = list(G.nodes(data=True))[5]
    n19 = (19, {'pos' : avg_pos(n6[1]['pos'],n11[1]['pos']),'type':'e'})
    G.add_nodes_from([n19])
    G.add_edges_from([(n6[0], n19[0]), (n19[0], n11[0])])
    
    print("Trying to use p4 on p3 graph")
    try:
        p4(G, 15, graph_layers.get_node_id_gen())
    except CannotExecuteProduction:
        print("CannotExecuteProduction exception caught")
    else:
        print("Production ran on invalid graph")
    
    print_res(graph_layers, lambda :p3(G, 15, graph_layers.get_node_id_gen()),'p3',2)
zad_2_tests()